# 1º División del conjunto de datos:
La división la he realizado con un 80% de los datos para train (8,929 registros) y un 20% para test (2,233 registros). Considero esta proporción adecuada para un dataset de este tamaño, ya que permite entrenar el modelo con una cantidad significativa de datos mientras se reserva un conjunto suficiente para evaluar su rendimiento.


In [1]:
# Lo primero que hago es importar pandas para trabajar con los datos y train_test_split de sklearn.model_selection para hacer la división de los datos.
import pandas as pd 
from sklearn.model_selection import train_test_split

# Creo un objeto que llamaré df donde con la función de pandas read.csv le indico la ruta del archivo.
df = pd.read_csv(r"C:\Users\javie\Downloads\bank_dataset.CSV")

# Efectúo la división de los datos que serán 80% para train y 20% para test.
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Para verificar esto hago un print de cada parte de la división.
print("Tamaño del conjunto de entrenamiento:", train_data.shape)
print("Tamaño del conjunto de prueba:", test_data.shape)


Tamaño del conjunto de entrenamiento: (8929, 17)
Tamaño del conjunto de prueba: (2233, 17)


# 2º Codificación de variables Categóricas:
He decidido utilizar One-Hot Encoding para las variables categóricas, ya que estas no muestran un orden específico que deba tenerse en cuenta. 
Esta técnica nos permite transformar las categorías en columnas binarias, facilitando el uso de estas variables en modelos de machine learning.


In [2]:
# Importo OneHotEncoder de sklearn.preprocessing para efectuar la codificación de las variables.
from sklearn.preprocessing import OneHotEncoder

# Busco las columnas categóricas que serán ='object'
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Aplico la función OneHotEncoder al conjunto de train y test.
train_data_encoded = pd.get_dummies(train_data, columns=categorical_columns, drop_first=True)
test_data_encoded = pd.get_dummies(test_data, columns=categorical_columns, drop_first=True)

# Me aseguro de que ambas tablas tienen las mismas columnas tras la codificación.
train_data_encoded, test_data_encoded = train_data_encoded.align(test_data_encoded, join='left', axis=1, fill_value=0)

# Visualizo los primeros registros a manera de comprobación.
train_data_encoded.head()


,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown,deposit_yes
3955,28.0,5741,10,1042,5,200,3,False,False,False,...,False,False,False,False,False,True,False,False,False,True
11150,34.0,355,21,314,3,-1,0,False,False,False,...,False,False,False,False,False,False,False,False,True,False
5173,48.0,201,10,594,1,91,1,False,False,False,...,False,False,False,True,False,False,False,True,False,True
3017,53.0,1961,15,351,1,-1,0,False,True,False,...,False,False,False,False,False,False,False,False,True,True
2910,53.0,1624,11,166,3,97,7,False,False,False,...,False,False,False,True,False,False,False,True,False,True


Se puede ver en el resultado que ahora se han codificado como True o False en las variables según correspondían , por ejemplo para el registro 3955 evelua todas las columns correspondientes a los meses poniendo únicamente True a la de septiembre y False para el resto . Esta codificación será interpretada por el modelo para poder trabajar con esta información sin tenen ningún tipo de "influencia". Esto lo digo porque debo reconocer que mi primera idéa era dar número aleatorios a las distintas variables pero despues de ver la teoría y tutoriales comprendí que esto puede afectar de manera negativa a como el modelo interpreta los datos y vamos, que sería un desastre.

# 3º Estandarización de las Variables Numéricas:
He decidido utilizar StandardScaler para la estandarización de las variables numéricas. Este método transforma los datos para que tengan una media de 0 y una desviación estándar de 1, lo que es especialmente útil cuando las características tienen diferentes escalas y magnitudes. La estandarización es una práctica común en el preprocesamiento de datos, ya que puede mejorar el rendimiento de muchos algoritmos de machine learning.


In [14]:
# Importo StandarScaler de sklearn.preprocessing para realizar la estandarización de las variables.
from sklearn.preprocessing import StandardScaler

# Busco las columnas numéricas del df.
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Creo objeto escalador
scaler = StandardScaler()

# Aplicar la estandarización en el conjunto de train y test.
train_data_encoded[numeric_columns] = scaler.fit_transform(train_data_encoded[numeric_columns])
test_data_encoded[numeric_columns] = scaler.transform(test_data_encoded[numeric_columns])

# Visualizo los datos estandarizados.
train_data_encoded[numeric_columns].head()


,age,balance,day,duration,campaign,pdays,previous
3955,-1.114015,1.346271,-0.666669,1.941942,0.921804,1.374037,0.991722
11150,-0.609359,-0.371778,0.635888,-0.162532,0.182456,-0.476209,-0.367601
5173,0.568172,-0.420902,-0.666669,0.646881,-0.556892,0.370670,0.085506
3017,0.988719,0.140511,-0.074597,-0.055574,-0.556892,-0.476209,-0.367601
2910,0.988719,0.033013,-0.548255,-0.590364,0.182456,0.425901,2.804154


Al aplicar StandardScaler, los valores de las variables numéricas se transforman para que tengan una media de 0 y una desviación estándar de 1. Esto es especialmente útil cuando las variables tienen diferentes escalas, ya que evita que algunas características dominen el modelo simplemente debido a sus magnitudes.

Ejemplo de Resultado: En Age tomando como referencia los registros: 3955, 11150 y 5173 teníamos :
Edad (original): 28, 34, 48
Edad (estandarizado): -1.11, -0.61, 0.57
Los valores numéricos se centran y normalizan, asegurando que cada variable tenga una influencia similar en el modelo, por lo que se puede ver que el registro 5173 es el que se acerca más a la media porque está más próximo a 0 mientras que el registro 3955 con -1.11 es el que más se aleja.
TENIENDO EN CUENTA QUE SOLO ESTAMOS TOMANDO ESTOS 3 REGISTROS PARA ESTA EXPLICACIÓN DE LOS RESULTADOS.